In [141]:
import sys
import os
import re
import pandas as pd
from datetime import datetime
from collections import defaultdict
from dataclasses import dataclass, field
from typing import Optional


# Caminho para o root do projeto
root_path = os.path.abspath("..")   # sobe 1 nível — ajuste se precisar

sys.path.append(root_path)

from database.db_connection import access_db_for_test

from romaneio import ProcessRomaneio, BuildDataFrames, convert_types, append_df_to_bq

In [142]:
client = access_db_for_test()

In [143]:
input_text = """
Romaneio 15/12
Total de notas 30
Realizadas 29
10 notas a $27,40..$275,00
20 notas a $36,50..$730,00
Total $968,50
Pedágio $27,40
Almoço $60,00
Café $20,00
Abastecimento $455,64
Ceps 030 ao 034.
"""

In [144]:
processa_romaneio = ProcessRomaneio(input_text, year=2026)

output = processa_romaneio.process()

output

RomaneioData(data='2026-12-15', total_notas=30, realizadas=29, notas_leves=10, notas_pesadas=20, fat_notas_leves=275.0, fat_notas_pesadas=730.0, valor_total=1005.0, ceps='30, 31, 32, 33, 34', despesas={'Pedágio': 27.4, 'Almoço': 60.0, 'Café': 20.0, 'Abastecimento': 455.64})

In [145]:
data_builder = BuildDataFrames(output)
df_receitas = data_builder.build_df_receitas()
df_despesas = data_builder.build_df_despesas()
df_peso_notas = data_builder.build_df_peso_notas()

In [146]:
df_receitas = convert_types(df_receitas, ["DATA", "CREATED_AT"])
df_despesas = convert_types(df_despesas, ["DATA", "CREATED_AT"])
df_peso_notas = convert_types(df_peso_notas, ["DATA", "CREATED_AT"])

In [147]:
df_receitas

,DATA,TOTAL_NOTAS,NOTAS_REALIZADAS,VALOR_TOTAL,CEPS,CREATED_AT
0,2026-12-15,30,29,1005.0,"30, 31, 32, 33, 34",2025-12-20


In [148]:
df_despesas

,DATA,CATEGORIA,VALOR,CREATED_AT
0,2026-12-15,Pedágio,27.40,2025-12-20
1,2026-12-15,Almoço,60.00,2025-12-20
2,2026-12-15,Café,20.00,2025-12-20
3,2026-12-15,Abastecimento,455.64,2025-12-20


In [149]:
df_peso_notas

,DATA,NOTAS_LEVES,NOTAS_PESADAS,FAT_NOTA_LEVE,FAT_NOTA_PESADA,CREATED_AT
0,2026-12-15,10,20,275.0,730.0,2025-12-20


In [150]:
append_df_to_bq(
    client,
    df_receitas,
    "SBOX_ISRAEL.RECEITAS"
)

c:\Users\DELL\OneDrive\Documentos\projetos\dashboard_israel\.venv\Lib\site-packages\google\cloud\bigquery\_pandas_helpers.py:484: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


In [151]:
append_df_to_bq(
    client,
    df_despesas,
    "SBOX_ISRAEL.DESPESAS"
)

c:\Users\DELL\OneDrive\Documentos\projetos\dashboard_israel\.venv\Lib\site-packages\google\cloud\bigquery\_pandas_helpers.py:484: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


In [152]:

append_df_to_bq(
    client,
    df_peso_notas,
    "SBOX_ISRAEL.PESO_NOTAS"
)

c:\Users\DELL\OneDrive\Documentos\projetos\dashboard_israel\.venv\Lib\site-packages\google\cloud\bigquery\_pandas_helpers.py:484: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(
